In [1]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
include("VoronoiGen.jl")
using BenchmarkTools

In [2]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [3]:
nx = ny = nz = 100

nGrains = 1000


#Maximum sound speed in the model
c = 8000
#Frequency, hz
f0=1e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = c/(8*f0)
#Maximum time increment
dt = dx/(c*sqrt(3))

baseMat = Main.EFIT.AnisoMats["X6CrNi1811"]
c = baseMat.c
ρ = baseMat.ρ
materials = Array{Main.EFIT.AnisoMat,1}()

Main.EFIT.AnisoMat[]

In [4]:
matGrid = fullRandomGen(nx,ny,nz,nGrains);

LoadError: UndefVarError: plt not defined

In [ ]:
for i in 1:nGrains+2
    rx = rand(Float32)*180
    ry = rand(Float32)*180
    rz = rand(Float32)*180

    mat = Main.EFIT.AnisoMat(ρ, Main.EFIT.rotateMatrix(c,rx,ry,rz))
    push!(materials, mat)
end

In [ ]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid.+1),materials,dt,dx);


#Create a source function from which input can be taken
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end
nframes = 200
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
fig,ax,plt = volume(Array(grid.vx),algorithm=:mip,colorrange = (0, 0.04),colormap=:curl, transparency=true)

#Step the simulation
function stepSim(t)
    println(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)
    #Apply the source to the Z-direction
    @parallel (45:55,45:55,98:98) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))

    #Update the plot
    plt.volume = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)


Creating grid of size 200, 200, 200


Initialized averaging dict, hashed 84989 material combinations



Dict{Int64, Matrix{Float32}} with 84989 entries:
  198722776688754709 => [4.05797f10 1.2236f10 … 4.0452f10 1.80484f7; 1.2236f10 …
  261493805356351652 => [4.24232f11 1.10663f11 … 1.50672f11 -2.35091f9; 1.10663…
  252205427447628145 => [9.83402f10 3.19828f10 … 1.97931f10 -1.3968f9; 3.19828f…
  268811098172752453 => [1.53673f11 8.15586f10 … 1.06756f11 7.17646f8; 8.15586f…
  179020816820797950 => [2.22356f11 9.67309f10 … 6.82998f10 -1.56911f10; 9.6730…
  192248620292636898 => [1.94178f10 1.48751f10 … 4.01874f10 1.23769f9; 1.48751f…
  124695317373518132 => [1.14562f11 8.13509f10 … 1.52301f11 -4.82118f9; 8.13509…
  215609922356051989 => [3.47162f10 1.19915f10 … 1.01734f10 -3.7832f9; 1.19915f…
  223494168089461336 => [2.61973f11 1.15517f11 … 4.33361f10 5.2723f9; 1.15517f1…
  101332537815793840 => [7.92566f10 1.95311f10 … 6.1568f10 4.39819f9; 1.95311f1…
  254175782367002940 => [1.08866f11 5.14123f10 … 9.99308f10 4.46985f9; 5.14123f…
  263182809786155083 => [1.46976f11 6.70874f10 … 9.71974f10 